<a href="https://colab.research.google.com/github/maqmal/Machine-Learning/blob/main/tubes_ann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [460]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
# %matplotlib inline

In [461]:
df = pd.read_csv('/content/Clean_data.csv')
df = df.drop(df.columns[0], axis=1) # Drop kolom index

# Memisah label dari feature untuk split data
label_y = df['Exited']
feature = df.drop(columns=['Exited'], axis=1)

# Pisahkan data test dan data train
X_train, X_test, Y_train, Y_test = train_test_split(feature, label_y, train_size=0.60)

# Normalisasi data menggunakan standard scaler
X_train = StandardScaler().fit_transform(X_train)
X_test = StandardScaler().fit_transform(X_test)

Y_train = Y_train.to_numpy()
Y_test = Y_test.to_numpy()


In [462]:
class ANN:
  def __init__(self, layers, learning_rate, epoch, bias):
    self.layers = layers
    self.learning_rate = learning_rate
    self.epoch = epoch
    self.networks = {}
    self.error_list = []
    self.data_x = None
    self.data_y = None
    self.bias = bias

  def sigmoid(self, x):
    return 1/(1+np.exp(-x))

  def sigmoid_der(self, x):
    return self.sigmoid(x)*(1-self.sigmoid(x))

  def set_weight(self):
    np.random.seed(1)
    self.networks['w1'] = np.random.rand(self.layers[0],self.layers[1])
    self.networks['b1'] = self.bias 
    self.networks['w2'] = np.random.rand(self.layers[1],self.layers[2])
    self.networks['b2'] = self.bias 
    self.networks['w3'] = np.random.rand(self.layers[2],self.layers[3])
    self.networks['b3'] = self.bias 

  def fit(self, x, y):
    self.data_x = x
    self.data_y = y

    self.set_weight()

    for i in range(self.epoch):

      V1 = np.dot(self.data_x, self.networks['w1']) + self.networks['b1']
      A1 = self.sigmoid(V1)

      V2 = np.dot(A1, self.networks['w2']) + self.networks['b2']
      A2 = self.sigmoid(V2)

      V3 = np.dot(A2, self.networks['w3']) + self.networks['b3']
      A3 = self.sigmoid(V3)

      E = self.data_y.reshape(6000,1) - A3
  
      MSE = (np.power(E,2).sum())/len(self.data_x)
      print(MSE)
      self.error_list.append(MSE)  

      der_A3 = self.sigmoid_der(A3)
      D3 =  der_A3 * E
      dW3 = A2.T.dot(D3)

      der_A2 = self.sigmoid_der(A2)
      D2 =  der_A2 * D3.dot(self.networks['w3'].T)
      dW2 = A1.T.dot(D2)

      der_A1 = self.sigmoid_der(A1)
      D1 = der_A1 * D2.dot(self.networks['w2'].T)
      dW1 = self.data_x.T.dot(D1)

      self.networks['w1'] += dW1 * self.learning_rate
      self.networks['w2'] += dW2 * self.learning_rate
      self.networks['w3'] += dW3 * self.learning_rate


  def predict(self, X):
    V1 = X.dot(self.networks['w1']) + self.networks['b1']
    A1 = self.sigmoid(V1)

    V2 = V1.dot(self.networks['w2']) + self.networks['b2']
    A2 = self.sigmoid(V2)

    V3 = V2.dot(self.networks['w3']) + self.networks['b3']
    A3 = self.sigmoid(V3)

    # for i in A3:
    #   print(i)
  
    return np.round(A3)

  def akurasi(self, y_actual, y_pred):
        akurasi = int(sum(y_actual == y_pred) / len(y_actual) * 100)
        return akurasi


  def plot_loss(self):
    # print(self.error_list)
    plt.plot(self.error_list)
    plt.xlabel("Iteration")
    plt.ylabel("Error")
    plt.title("Error curve for training")
    # plt.show()  



In [ ]:
artificial_neural_network = ANN(layers=[17,8,6,1], learning_rate=0.005, epoch=100, bias=0.45)
artificial_neural_network.fit(X_train,Y_train)
artificial_neural_network.plot_loss()

train_pred = artificial_neural_network.predict(X_train)
test_pred = artificial_neural_network.predict(X_test)

train_pred = [int(i) for i in train_pred]
test_pred = [int(i) for i in test_pred]

# for i in range(len(test_pred)):
#   print("Data ke-{} = {}".format(i,test_pred[i]))

print("Akurasi Data Training {}".format(round(artificial_neural_network.akurasi(Y_train, train_pred),1)))
print("Akurasi Data Test {}".format(round(artificial_neural_network.akurasi(Y_test, test_pred),1)))
F1 = round(f1_score(Y_test, test_pred, average='weighted'),2)*100
print("F1-Score {}%".format(F1))